In [64]:
import aocpy as aoc
from aocpy import print_bools, print_mat, add_tuple
import re
from enum import Enum
from copy import deepcopy
from pprint import pprint
from functools import reduce, cmp_to_key
from itertools import groupby, combinations, permutations
from time import time
from queue import PriorityQueue
from math import prod, ceil, comb, perm
from collections import defaultdict
from dataclasses import dataclass
from bisect import insort
import numpy as np
import string

aoc.init(day=22, year=2022)


Select(description='Input file:', index=1, options=('input', 'example'), rows=4, value='example')

## Parse file

In [70]:
lines = aoc.get_lines()

N = 4 if aoc.box.value == "example" else 50
pat = re.compile(r"(L|R)")
instructions = pat.split(lines[-1])


Done reading "input"


### Part 1

In [71]:
w = max(map(len, lines[:-2]))
h = len(lines[:-2])

grid = np.zeros((h, w), dtype=bool)
empty = np.ones((h, w), dtype=bool)
D = np.full((h, w), " ", dtype=str)

START = None
for y, line in enumerate(lines[:-2]):
    for x, c in enumerate(line):
        if c == ".":
            if y == 0 and START is None:
                START = (y, x)
                D[y, x] = ">"
            grid[y, x] = True
        if c == "." or c == "#":
            empty[y, x] = False
        D[y, x] = c


In [72]:
r, c = START
a = 0

for instr in instructions:
    match instr:
        case "L":
            a = (a - 1) % 4
        case "R":
            a = (a + 1) % 4
        case _:
            n = int(instr)
            for _ in range(n):
                y, x = (r, c)
                match a:
                    case 0:
                        x = (x + 1) % w
                    case 1:
                        y = (y + 1) % h
                    case 2:
                        x = (x - 1) % w
                    case 3:
                        y = (y - 1) % h

                if empty[y, x]:
                    match a:
                        case 0:
                            x = np.where(empty[y, :] == False)[0][0]
                        case 1:
                            y = np.where(empty[:, x] == False)[0][0]
                        case 2:
                            x = np.where(empty[y, :] == False)[0][-1]
                        case 3:
                            y = np.where(empty[:, x] == False)[0][-1]

                if grid[y, x]:
                    r, c = (y, x)
                else:
                    break

print(1000 * (r+1) + 4 * (c+1) + a)


31568


### Part 2

In [73]:
from scipy.spatial.transform import Rotation as R

debug = []
cube = []
faces = []
cube_to_grid = []
for i in range(0, h//N):
    for j in range(0, w//N):
        if not empty[i * N, j * N]:
            cube.append(grid[i*N:(i+1)*N, j*N:(j+1)*N])
            debug.append(D[i*N:(i+1)*N, j*N:(j+1)*N])
            faces.append((j, i))
            cube_to_grid.append((i*N, j*N))


stack = [(0, np.array([0, 1, 0]), np.array([0, 0, -1]))]
V = set()
map2d_to_3d = [{} for _ in range(6)]
map3d_to_2d = [{} for _ in range(6)]
while len(stack) > 0:
    i, normal, up = stack.pop()
    if i in V:
        continue
    V.add(i)

    left = np.cross(normal, up)
    for r in [0, N-1]:
        for c in [0, N-1]:
            x = -N if c == 0 else N
            y = N if r == 0 else -N
            coord = normal * N + -x * left + y * up
            coord = tuple(map(round, coord.tolist()))
            u = tuple(up.astype(int).tolist())
            n = tuple(normal.astype(int).tolist())
            map2d_to_3d[i][(r, c)] = coord
            map3d_to_2d[i][coord] = (r, c)

    x, y = faces[i]
    f = (x + 1, y)
    if f in faces:
        j = faces.index(f)
        rot = R.from_rotvec(np.radians(90) * up)
        n = rot.apply(normal).astype(int)
        stack.append((j, n, up))
    f = (x - 1, y)
    if f in faces:
        j = faces.index(f)
        rot = R.from_rotvec(np.radians(-90) * up)
        n = rot.apply(normal).astype(int)
        stack.append((j, n, up))
    f = (x, y + 1)
    if f in faces:
        j = faces.index(f)
        rot = R.from_rotvec(np.radians(-90) * left)
        n = rot.apply(normal).astype(int)
        u = rot.apply(up).astype(int)
        stack.append((j, n, u))


M = [{} for _ in range(6)]
for i in range(6):
    for j in range(6):
        if i == j:
            continue

        Mi = map2d_to_3d[i]
        Mj = map2d_to_3d[j]
        Pi = []
        Pj = []
        for p2d, p3d in Mi.items():
            for q2d, q3d in Mj.items():
                if p3d == q3d:
                    Pi.append(p2d)
                    Pj.append(q2d)
        if len(Pi) == 0:
            continue

        pi1 = Pi[0]
        pi2 = Pi[1]
        pj1 = Pj[0]
        pj2 = Pj[1]

        vi = pi1[1] == pi2[1]
        di = np.array([0, 1]) if vi else np.array([1, 0])
        di = -di if pi1[int(vi)] == 0 else di

        dx = np.sign(pi2[1] - pi1[1])
        X = [pi1[1]]*N if dx == 0 else range(
            pi1[1], pi2[1] + dx, dx)
        dy = np.sign(pi2[0] - pi1[0])
        Y = [pi1[0]] * N if dy == 0 else range(
            pi1[0], pi2[0] + dy, dy)
        interp_i = list(zip(Y, X))

        ai = None
        match di.tolist():
            case [0, 1]:
                ai = 2
            case [1, 0]:
                ai = 3
            case [0, -1]:
                ai = 0
            case [-1, 0]:
                ai = 1

        vj = pj1[1] == pj2[1]
        dj = np.array([0, 1]) if vj else np.array([1, 0])
        dj = -dj if pj1[int(vj)] == 0 else dj

        dx = np.sign(pj2[1] - pj1[1])
        X = [pj1[1]]*N if dx == 0 else range(
            pj1[1], pj2[1] + dx, dx)
        dy = np.sign(pj2[0] - pj1[0])
        Y = [pj1[0]] * N if dy == 0 else range(
            pj1[0], pj2[0] + dy, dy)
        interp_j = list(zip(Y, X))

        aj = None
        match dj.tolist():
            case [0, 1]:
                aj = 2
            case [1, 0]:
                aj = 3
            case [0, -1]:
                aj = 0
            case [-1, 0]:
                aj = 1

        M[i] |= {tuple(xi + di): (j, aj, tuple(xj))
                 for xi, xj in zip(interp_i, interp_j)}
        M[j] |= {tuple(xj + dj): (i, ai, tuple(xi))
                 for xi, xj in zip(interp_i, interp_j)}


In [74]:
(r, c) = (0, 0)
f = 0
a = 0

for instr in instructions:
    match instr:
        case "L":
            a = (a - 1) % 4
        case "R":
            a = (a + 1) % 4
        case _:
            n = int(instr)
            for _ in range(n):
                debug[f][r, c] = [">", "v", "<", "^"][a]
                y, x = (r, c)
                fnew = f
                anew = a
                match a:
                    case 0:
                        x = x + 1
                    case 1:
                        y = y + 1
                    case 2:
                        x = x - 1
                    case 3:
                        y = y - 1

                if (y, x) in M[f]:
                    fnew, anew,  (y, x) = M[f][(y, x)]
                if cube[fnew][y, x]:
                    r, c = (y, x)
                    f = fnew
                    a = anew
                else:
                    break


y, x = cube_to_grid[f]
print(1000 * (y+r+1) + 4 * (x+c+1) + a)


36540
